In [4]:
from __future__ import print_function, division

from keras.datasets import mnist
#from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Activation, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import Concatenate
from keras.layers.advanced_activations import ReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.models import Sequential, Model
from keras.optimizers import Adam, Adadelta
from keras import losses, activations
from keras.utils import to_categorical
import keras.backend as K
import scipy

import matplotlib.pyplot as plt

import numpy as np

from dataread import DataRead

In [4]:
def weightedMSELoss(y_true, y_pred):
    masked_difference = np.multiply(y_true[:,:,:,:,3] * (y_true[:,:,:,:,:3] - y_pred))
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)

class GANScape():
    def __init__(self):
        self.imgHeight = 64
        self.imgWidth = 64
        self.channels = 4
        self.imgShape = (self.imgHeight, self.imgWidth, self.channels)
        self.mask_height = 10
        self.mask_width = 10
        self.num_classes = 10
        self.weightForJointLossInGenerator = 0.5
        
        optimizer = Adadelta()
        
        self.generatorWithMSE, self.generatorWithJointLoss = self.buildGenerator() 
        self.discriminator = self.buildDiscriminator()
        
        self.combinedForPhaseTwo = Model(self.generator, self.discriminator)
        
        #compile network
        self.generatorWithMSE.compile(optimizer=optimizer, loss=weightedMSELoss)
        self.generatorWithJointLoss.compile(optimizer=optimizer, loss=['binary_crossentropy', weightedMSELoss], loss_weights=[self.weightForJointLossInGenerator, 1-self.weightForJointLossInGenerator])
        self.discriminator.compile(optimizer=optimizer, loss=['binary_crossentropy'])
        
        """

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['mse', 'categorical_crossentropy'],
            loss_weights=[0.5, 0.5],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        

        # The generator takes noise as input and generates imgs
        masked_img = Input(shape=self.img_shape)
        gen_img = self.generator(masked_img)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid, _ = self.discriminator(gen_img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(masked_img , valid)
        self.combined.compile(loss=['mse'],
            optimizer=optimizer)
        """

        


    def buildGenerator(self):
        genInput = Input(shape=self.imgShape)
        
        gen = Conv2D(64, kernel_size=(5, 5), padding='same')(genInput)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        
        gen = Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same')(gen)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        gen = Conv2D(128, kernel_size=(3, 3), padding='same')(gen)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        
        gen = Conv2D(256, kernel_size=(3, 3), strides=(2,2), padding='same')(gen)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        gen = Conv2D(256, kernel_size=(3, 3), padding='same')(gen)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        gen = Conv2D(256, kernel_size=(3, 3), padding='same')(gen)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        
        gen = Conv2D(256, kernel_size=(3, 3), dilation_rate=(2, 2), padding='same')(gen)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        gen = Conv2D(256, kernel_size=(3, 3), dilation_rate=(4, 4), padding='same')(gen)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        gen = Conv2D(256, kernel_size=(3, 3), dilation_rate=(8, 8), padding='same')(gen)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        gen = Conv2D(256, kernel_size=(3, 3), dilation_rate=(16, 16), padding='same')(gen)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        
        gen = Conv2DTranspose(128, kernel_size=(4, 4), strides=(2, 2), padding='same')(gen)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        gen = Conv2D(128, kernel_size=(3, 3), padding='same')(gen)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        
        gen = Conv2DTranspose(64, kernel_size=(4, 4), strides=(2, 2), padding='same')(gen)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        gen = Conv2D(32, kernel_size=(3, 3), padding='same')(gen)
        gen = ReLU()(gen)
        gen = BatchNormalization()(gen)
        
        gen = Conv2D(3, kernel_size=(3, 3), padding='same')(gen)
        genOutput = Activation(activations.sigmoid)(gen)
        
        return Model(genInput, genOutput), Model(genInput, genOutput)
       
        
    def buildDiscriminator(self):

        globalDiscInput = Input(shape=self.imgShape)
        globalDisc = Conv2D(64, kernel_size=(5, 5), strides=(2,2), padding='same')(globalDiscInput)
        globalDisc = ReLU()(globalDisc)
        globalDisc = BatchNormalization()(globalDisc)
        globalDisc = Conv2D(128, kernel_size=(5, 5), strides=(2,2), padding='same')(globalDisc)
        globalDisc = ReLU()(globalDisc)
        globalDisc = BatchNormalization()(globalDisc)
        globalDisc = Conv2D(256, kernel_size=(5, 5), strides=(2,2), padding='same')(globalDisc)
        globalDisc = ReLU()(globalDisc)
        globalDisc = BatchNormalization()(globalDisc)
        globalDisc = Conv2D(512, kernel_size=(5, 5), strides=(2,2), padding='same')(globalDisc)
        globalDisc = ReLU()(globalDisc)
        globalDisc = BatchNormalization()(globalDisc)
        globalDisc = Conv2D(512, kernel_size=(5, 5), strides=(2,2), padding='same')(globalDisc)
        globalDisc = ReLU()(globalDisc)
        globalDisc = BatchNormalization()(globalDisc)
        globalDisc = Conv2D(512, kernel_size=(5, 5), strides=(2,2), padding='same')(globalDisc)
        globalDisc = ReLU()(globalDisc)

        globalDisc = Flatten()(globalDisc)
        globalDisc = Dense(1024)(globalDisc)
        
        
        
        localDiscInput = Input(shape=self.imgShape)
        localDisc = Conv2D(64, kernel_size=(5, 5), strides=(2,2), padding='same')(localDiscInput)
        localDisc = ReLU()(localDisc)
        localDisc = BatchNormalization()(localDisc)
        localDisc = Conv2D(128, kernel_size=(5, 5), strides=(2,2), padding='same')(localDisc)
        localDisc = ReLU()(localDisc)
        localDisc = BatchNormalization()(localDisc)
        localDisc = Conv2D(256, kernel_size=(5, 5), strides=(2,2), padding='same')(localDisc)
        localDisc = ReLU()(localDisc)
        localDisc = BatchNormalization()(localDisc)
        localDisc = Conv2D(512, kernel_size=(5, 5), strides=(2,2), padding='same')(localDisc)
        localDisc = ReLU()(localDisc)
        localDisc = BatchNormalization()(localDisc)
        localDisc = Conv2D(512, kernel_size=(5, 5), strides=(2,2), padding='same')(localDisc)
        localDisc = ReLU()(localDisc)
        
        localDisc = Flatten()(localDisc)
        localDisc = Dense(1024)(localDisc)
        
        
        concatedDisc = Concatenate()([globalDisc, localDisc])
        concatedDisc = Dense(2)(concatedDisc)
        
        return Model(inputs=[globalDiscInput, localDiscInput], outputs=concatedDisc), Model(inputs=[globalDiscInput, localDiscInput], outputs=concatedDisc)


    def train(self, epochs, batch_size=128, phaseOneRatio, phaseTwoRatio, numOfAllTrainDataSample, sample_interval=50):

        trainData = DataRead(r'C:\\places\\','train',25,5)#path, mode, batch size, batch num  folder_path, data_type, batch_size, batch_num)
        
        trainData.
        
        
        
        
        for epoch in range(epochs):
            tEnd = numOfTrainDataSamplesPerEpoch // batch_size
            tC = int(phaseOneRatio * TEnd)
            tD = int(phaseTwoRatio * TEnd)
            t = 0
            while t < tEnd:
                if t < tC:
                    # ---------------------
                    #  PHASE ONE, train only Generatort with weighted MSE
                    # ---------------------
                    self.generatorWithMse.train_on_batch()
                    
                    
                else if t < (tC + tD):
                    # ---------------------
                    #  PHASE TWO, train only Discriminator with binary crossentropy
                    # ---------------------
                    generatorOutput = self.generatorWithMse.predict()
                    self.discriminator.train_on_batch(generatorOutput)
                    self.discriminator.train_on_batch(realImgs)
                
                else:
                    # ---------------------
                    #  PHASE THREE, train Generator with joint loss + Discriminator with binary crossentropy
                    # ---------------------
                    generatorOutput = self.generatorWithJointLoss.predict()
                    discriminatorOutput = self.discriminator.predict()
                    self.discriminator.train_on_batch(generatorOutput)
                    self.discriminator.train_on_batch(realImgs)
                    self.generatorWithJointLoss.train_on_batch(discriminatorOutput)


SyntaxError: invalid syntax (<ipython-input-4-395a2f7bee7a>, line 174)

In [ ]:
n = Network()

In [ ]:
n.generator.summary()

In [ ]:
n.discriminator.summary()

In [6]:
train = DataRead(r'/media/gamma/TOSHIBA EXT/DL/places_64x64/','train',25,5)#path, mode, batch size, batch num  folder_path, data_type, batch_size, batch_num)

172703
172703
172703


In [45]:
import cv2

In [ ]:
#a = cv2.imread('/media/gamma/TOSHIBA EXT/DL/1.png')
a = cv2.imread(r'\\media\\gamma\\TOSHIBA EXT\\DL\\1.png')
a.shape
cv2.imshow('img',a)
cv2.waitKey(0)
cv2.destroyAllWindows()